In [2]:
from math import ceil
import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import write
from synthesizing_new import Tone, Synthesis

In [3]:
refrain =  np.matrix([
        [220, 200.2, 100.2, 200],
        [  0, 300.2,   150, 400],
        [  0, 600.3,   250, 600]
    ])
durations = [0.5, 0.5, 0.5, 0.5]
sample_rate = 22050
refrain.shape

(3, 4)

In [4]:
S = Synthesis(refrain, durations, sample_rate)

In [5]:
durations_in_samples = S.durations_in_samples
max_durations_in_samples = S.max_durations_samples
frequencies = S.refrain
total_duration_in_samples = S.total_duration_in_samples

In [6]:
def generate_tone(frequency, duration_in_samples, amplitude=0.5, pad_amount=0):
    each_sample = np.arange(duration_in_samples)
    
    sine = np.sin(2 * np.pi * each_sample * frequency / sample_rate) * amplitude
   
    if pad_amount - duration_in_samples > 0:
        pad_for_each_side = (pad_amount - duration_in_samples) / 2
        beginning = int(pad_for_each_side)
        end = ceil(pad_for_each_side)
        return np.pad(sine, (beginning, end)) # note: this will also work for rests
    
    return sine

In [17]:
# add these to synthesizing_new
cumsum_max_durations = np.ravel(np.cumsum(max_durations_in_samples))
sample_boundaries = np.insert(cumsum_max_durations[:-1], 0, 0)

array([    0, 11025, 22045, 33070])

In [7]:
out_matrix = np.zeros((refrain.shape[0], total_duration_in_samples))

### note May 20, 5:50pm - I need to cumulative sum max_durations_in_samples. 
# Like I do with sample_boundaries in the og synthesising module.
# That's why I'm running into a problem iterating and assigning tones to out_matrix.

for i,r in np.ndenumerate(refrain): 
    print(i)
    print(max_durations_in_samples[0,i[1]])
    tone = generate_tone(r, durations_in_samples[i], pad_amount=max_durations_in_samples[0,i[1]])
    if i == (0, 0):
        print(f'axis0 = {i[0]}\naxis1 = 0 to {tone.size}\n')
        out_matrix[i[0],0:tone.size] = tone
    else:
        print(f'axis0 = {i[0]}\naxis1 = {max_durations_in_samples[0,i[1]]} to {max_durations_in_samples[0,i[1]] + tone.size}\n')
        out_matrix[i[0], max_durations_in_samples[0,i[1]] : max_durations_in_samples[0,i[1]] + tone.size] = tone

(0, 0)
11025
axis0 = 0
axis1 = 0 to 11025

(0, 1)
11020
axis0 = 0
axis1 = 11020 to 22040

(0, 2)
11025
axis0 = 0
axis1 = 11025 to 22050

(0, 3)
11025
axis0 = 0
axis1 = 11025 to 22050

(1, 0)
11025
axis0 = 1
axis1 = 11025 to 22050

(1, 1)
11020
axis0 = 1
axis1 = 11020 to 22040

(1, 2)
11025
axis0 = 1
axis1 = 11025 to 22050

(1, 3)
11025
axis0 = 1
axis1 = 11025 to 22050

(2, 0)
11025
axis0 = 2
axis1 = 11025 to 22050

(2, 1)
11020
axis0 = 2
axis1 = 11020 to 22040

(2, 2)
11025
axis0 = 2
axis1 = 11025 to 22050

(2, 3)
11025
axis0 = 2
axis1 = 11025 to 22050



matrix([[0]])

In [8]:
for i,r in np.ndenumerate(refrain):
    print(i, r)
    
print(refrain)    

print(max_durations_in_samples)

(0, 0) 220.0
(0, 1) 200.2
(0, 2) 100.2
(0, 3) 200.0
(1, 0) 0.0
(1, 1) 300.2
(1, 2) 150.0
(1, 3) 400.0
(2, 0) 0.0
(2, 1) 600.3
(2, 2) 250.0
(2, 3) 600.0
[[220.  200.2 100.2 200. ]
 [  0.  300.2 150.  400. ]
 [  0.  600.3 250.  600. ]]
[[11025 11020 11025 11025]]


In [33]:
summed_audio = np.sum(out_matrix, axis=0)
summed_audio.shape

(44095,)

In [32]:
write('recycling/worksheet.wav', sample_rate, summed_audio)